In [30]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker
import pandas as pd
from src.utils import numeric_statistics

# 1.0 Get Data 

In [31]:
host = 'interview-2.ck1h5ksgzpiq.us-east-1.rds.amazonaws.com'  
porta = 3306
usuario = 'hotinterview'
senha = '6cT4jk9QWPhQC9KXWKDd'
schema = 'innodb'
tabela = 'sales_data'

url_conexao = f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{schema}'

In [32]:
engine = create_engine(url_conexao, echo=True)  # Use "echo=True" para ver as mensagens de log

In [33]:
Session = sessionmaker(bind=engine)
session = Session()
session.close()

In [34]:
query = 'SELECT * FROM sales_data'
df = pd.read_sql(query, engine)

2023-05-31 11:24:06,248 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-31 11:24:06,256 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 11:24:06,481 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-31 11:24:06,486 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 11:24:06,605 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-31 11:24:06,606 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 11:24:06,833 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 11:24:06,835 INFO sqlalchemy.engine.Engine DESCRIBE `innodb`.`SELECT * FROM sales_data`
2023-05-31 11:24:06,837 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 11:24:06,948 INFO sqlalchemy.engine.Engine SELECT * FROM sales_data
2023-05-31 11:24:06,950 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 11:25:22,789 INFO sqlalchemy.engine.Engine ROLLBACK


Para não ficar fazendo consultas ao banco de dados todas as vezes que abrir esse notebook, decidi salvar
esses dados como um arquivo .csv nesse path `src/data/data.csv`. Mas não subirei esse arquivo para o GitHub,
pois não são dados públicos. Ou seja, só quem tem acesso é de fato quem tem acesso ao banco de dados.

In [35]:
# path = "src/data/data.csv"
# df.to_csv(path)

In [36]:
df = pd.read_csv(path)

# 2.0 Observações iniciais 

- purchase_id: Identificação da compra na Hotmart;
- product_id: Identificação do produto na Hotmart;
- affiliate_id: Identificação do afiliado na Hotmart;
- producer_id: Identificação do produtor na Hotmart;
- buyer_id: Identificação do comprador na Hotmart;
- purchase_date: Data e hora em que a compra foi realizada;
- product_creation_date: Data e hora em que o produto foi criado na Hotmart;
- product_category: categoria do produto na Hotmart. Exemplo: e-book,
software, curso online, e-tickets, etc.;
- product_niche: nicho de mercado que o produto faz parte. Exemplo:
educação, saúde e bem-estar, sexualidade, etc.;
- purchase_value: valor da compra. Esse dado, assim como nicho e categoria foi
codificado para manter a confidencialidade. O valor apresentado no dataset é
o z-score do valor real;
- affiliate_commission_percentual: percentual de comissão que o afiliado
receberá da compra;
- purchase_device: tipo de dispositivo utilizado no momento da compra, como:
Desktop, Mobile, Tablet, ou Outros;
- purchase_origin: endereço do site do qual a pessoa veio antes da compra. Por
exemplo, se uma pessoa veio do Facebook, Youtube, ou até mesmo de outra
página no site oficial do produto;
- is_origin_page_social_network: informa se essa compra veio de uma URL do
Facebook, Youtube, Instagram, Pinterest, ou Twitter.

## Algumas regras de negócio
- Quando a compra for feita diretamente pelo produtor, ou seja, quando não
houver afiliado intermediando a compra, o campo
affiliate_commission_percentual terá valor 0, e o campo affiliate_id será
igual ao producer_id;
- No campo purchase_origin nós apenas consideramos o host do site. Isso quer
dizer que, se uma pessoa veio do site www.meuproduto.com/promocoes,
esse campo só irá retornar o valor www.meuproduto.com;

Validação da primeira regra de negócio. Nesse caso não temos nenhum registro onde a compra foi feita
diretamente pelo produtor.

In [6]:
df[df['product_id'] == df['affiliate_id']]

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,Venda


A coluna **purchase_origin** não possui um comportamento de host como **www.meuproduto.com**, mas irei considerar que a informação é essa mesmo. 

In [18]:
df['purchase_origin'].value_counts()

purchase_origin
Origin ef2b    330077
Origin 5187    167028
Origin adf0     77857
Origin 18eb     28693
Origin 3ade     13069
                ...  
Origin e51d         1
Origin d5eb         1
Origin 00ca         1
Origin 7475         1
Origin 290d         1
Name: count, Length: 9603, dtype: int64

In [25]:
numeric_statistics(df)

,type,Unique_Values,Mean,Median,Std,Min,Max,Range,Skew,Kurtosis
0,float64,10,0.345106,0.264265,0.285324,0.003009,0.943589,0.940580,1.027361,0.731008
1,float64,10,0.478060,0.427820,0.326569,0.028562,0.947067,0.918505,0.274694,-1.283361
2,float64,10,0.608442,0.609995,0.248933,0.283783,0.960927,0.677144,0.058754,-1.147297
3,float64,10,0.632378,0.623229,0.232442,0.166794,0.979037,0.812243,-0.502208,0.754721
4,float64,10,0.541905,0.589338,0.256004,0.147459,0.885584,0.738125,-0.477016,-1.021316


# DATA CLEANING 

In [21]:
df['affiliate_commission_percentual'].value_counts()

affiliate_commission_percentual
0.00     1338727
50.00      70931
60.00      33794
30.00      20419
40.00      19606
          ...   
29.60          1
29.48          1
39.03          1
30.11          1
28.88          1
Name: count, Length: 279, dtype: int64